In [1]:
import os
import PIL
import PIL.Image
import pathlib
import cv2
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline


import imgaug as ia
from imgaug import augmenters as iaa

from pathlib import Path

import keras
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


In [2]:
 def downscale_images(images, scale=30):
    """
    Downscales images in an array.
    Args:
        images: array of images to be downscaled 
        scale: 

    Returns: array of downscaled down images

    """
    resized = []
    
    for img in images:
        
        width = int(img.shape[1] * scale / 100)
        height = int(img.shape[0] * scale / 100)
        dim = (width, height)

        resized.append(cv2.resize(img, dim, interpolation = cv2.INTER_AREA))
    
    return resized


In [3]:
def bgr_to_rgb(images):
    return [cv2.cvtColor(i, cv2.COLOR_BGR2RGB) for i in images ]

In [4]:
def load_paintings(path):
    
    # directory with the images
    data_dir = pathlib.Path(path)
    
    
    # how many images are in the directory
    image_count = len(list(data_dir.glob('*')))
    print(f"Number of images found in {path}: {image_count}")


    paintings_path = list(data_dir.glob('*'))
    paintings_path = [str(path) for path in paintings_path]
    
    paintings_filenames = [path.split('/')[-1] for path in paintings_path]
    
    # loading images, in BGR
    paintings = [cv2.imread(painting_path) for painting_path in paintings_path]
    paintings = bgr_to_rgb(paintings)
        
    return paintings_filenames, paintings


In [ ]:
# the directory with paintings to augment
to_augment_dir = "../datasets/Tate-all-oil-canvas/"

# load the paintings, including their filenames (new filename is prefix+old filename)
paintings_filenames, paintings = load_paintings(to_augment_dir)

Number of images found in ../datasets/Tate-all-oil-canvas/: 3009


# Data Augmentation

## Imgaug

In [ ]:
def compare_orig_aug(orig, aug, limit = 15):
    
    for i in range(0, len(orig)): 
        ia.imshow(np.hstack([orig[i], aug[i]]))
        if i >= limit:
            break

In [ ]:
def get_augmented(images, augmenters_list):
    """
    Augments each image in an input array once. 
    
    Args: 
        images: images to be augmented

    Returns: an array of augmented images, size of an output array = size of input array

    """
    
    seq = iaa.Sequential(augmenters_list, random_order = True)
    
    augmented = seq(images=images)
        
    return augmented
    


### Augmenters list
This list defines the augmenters which will be used in a sequential/random order.

In [ ]:
# specify the augmentors to be used
augmenters_list = [
    
      
    iaa.GaussianBlur(sigma=(0, 3.0)), # blur images with a sigma of 0 to 3.0
   
    iaa.Sometimes(0.25,
                iaa.JpegCompression(compression=(0, 30)) ),# degrade the quality of images by JPEG-compressing them.  
        
    iaa.Affine(
        #scale={"x": (0.8, 1.2), "y": (0.8, 1.2)}, # zooming
        translate_percent={"x": (-0.05, 0.05), "y": (-0.05, 0.05)}, # shifting on axis
        rotate=(-20, 20), # rotating
        ),

    iaa.Crop(px=(0, 16)), # crop images from each side by 0 to 16px (randomly chosen)
    iaa.LinearContrast((0.75, 1.5)),
        
    iaa.Sometimes(0.25,
        iaa.Cutout(fill_mode="gaussian", fill_per_channel=True) ), # cutout a square, fill it with noise
                                                                   # algo should not rely on just one area of keypoints
        
        
    ]

In [ ]:
# augment the images
augmented = get_augmented(paintings, augmenters_list)

In [ ]:
#compare_orig_aug(paintings, augmented)

In [ ]:
def save_images(dest_dir, paintings_filenames, images):
    
    # create the directory where the augmented images will be stored
    dir_path = Path.cwd() / dest_dir
    dir_path.mkdir(parents=True, exist_ok=True)
    
    for filename, image in zip(paintings_filenames, images):
        
        filename = "aug_" + filename
        
        path = Path.cwd() / dest_dir / filename
        path = str(path)
        
        im_bgr = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        cv2.imwrite(path, im_bgr)
        
    print("Saving finished.")

            

In [ ]:
# make a new folder in this directory with "original_name"+ "_aug" postfix
aug_dir = to_augment_dir.split('/')[-2] + "_aug/"

# save the augmented paintings in the folder
save_images(aug_dir, paintings_filenames, augmented)
print(aug_dir)